In [16]:
import sys
!{sys.executable} -m pip install tensorflow

     |████████████████████████████████| 239.1 MB 15 kB/s              ta 0:00:29 eta 0:00:02
  Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
     |████████████████████████████████| 1.2 MB 31.8 MB/s            
  Using cached libclang-16.0.6-py2.py3-none-macosx_10_9_x86_64.whl (24.5 MB)
  Using cached tensorboard-2.15.1-py3-none-any.whl (5.5 MB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     |████████████████████████████████| 20.6 MB 37.7 MB/s            
  Using cached protobuf-4.25.1-cp37-abi3-macosx_10_9_universal2.whl (394 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached tensorflow_estimator-2.15.0-py2.py3-none-any.whl (441 kB)
     |████████████████████████████████| 1.7 MB 33.8 MB/s            
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     |████████████████████████████████| 3.2 MB 22.2 MB/s          

In [19]:
pip install tensorflow

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.3.2/libexec/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-11-16 09:32:23.770092: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [31]:
df = pd.read_csv("data/train_monthly_naturalized_flow.csv")
df = pd.get_dummies(df, columns=['site_id'])

In [32]:
df.head()

,forecast_year,year,month,volume,site_id_animas_r_at_durango,site_id_boise_r_nr_boise,site_id_boysen_reservoir_inflow,site_id_colville_r_at_kettle_falls,site_id_detroit_lake_inflow,site_id_dillon_reservoir_inflow,...,site_id_pueblo_reservoir_inflow,site_id_ruedi_reservoir_inflow,site_id_skagit_ross_reservoir,site_id_snake_r_nr_heise,site_id_stehekin_r_at_stehekin,site_id_sweetwater_r_nr_alcova,site_id_taylor_park_reservoir_inflow,site_id_virgin_r_at_virtin,site_id_weber_r_nr_oakley,site_id_yampa_r_nr_maybell
0,1911,1911,1,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1911,1911,2,85.071,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1911,1911,3,121.825,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1911,1911,4,224.172,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1911,1911,5,748.602,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

# Assuming df is your DataFrame with the necessary data

def create_features(df, issue_date_month, forecast_year):
    # Filter data for the forecast year up to the issue date
    current_year_data = df[(df['forecast_year'] == forecast_year) & (df['month'] < issue_date_month)]
    current_year_total = df[(df['forecast_year'] == forecast_year) & 
                            (df['month'] >= 4) & (df['month'] <= 7)].rename(columns = {"volume":"final_volume"})
    # Historical April to July data from previous years
    historical_data = df[(df['forecast_year'] < forecast_year) & (df['month'].between(4, 7))]
    historical_sum = historical_data.groupby(['site_id', 'forecast_year'])['volume'].sum().reset_index()

    # Combine current year data with historical data
    combined_data = current_year_data.merge(historical_sum, on='site_id', 
                                            how='left').merge(current_year_total, on='site_id', how='left')
    

    # More feature engineering can be done here if needed

    return combined_data

def create_lstm_model(input_shape):
    # LSTM model definition
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(50))
    model.add(Dense(25))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def train_and_predict(df, issue_date_month, forecast_year):
    features = create_features(df, issue_date_month, forecast_year)

    # Splitting features into X (inputs) and y (label)
    X = features.drop('final_volume', axis=1)  # Assuming 'volume' is the label
    y = features['final_volume']
    X = X.astype('float32')
    y = y.astype('float32')
    # Data splitting
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Reshape input for LSTM
    X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))

    # Model training
    model = create_lstm_model((X_train.shape[1], X_train.shape[2]))
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

    # Predictions
    predictions = model.predict(X_test)
    # Evaluate model performance

    return model, predictions

# Example usage
# df = pd.read_csv('your_data.csv')
# model, predictions = train_and_predict(df, issue_date_month=2, forecast_year=2023)


In [34]:
features = create_features(df, 5, 2022)

# Splitting features into X (inputs) and y (label)
X = features.drop('final_volume', axis=1)  # Assuming 'volume' is the label
y = features['final_volume']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X

KeyError: 'site_id'

In [ ]:
model, predictions, scaler = train_and_predict(df, issue_date_month=5, forecast_year = 2022)  # For example, May as issue date

ValueError: could not convert string to float: 'hungry_horse_reservoir_inflow'